In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import csv
import os
from keras import backend as K
import math
from scipy.stats import norm
from sklearn.ensemble import RandomForestRegressor
import os
from scipy.spatial.transform import Rotation as R
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#/content/drive/My Drive

#for x in os.listdir("/content/drive/My Drive/rachel rotated data"):
#    data = np.load("/content/drive/My Drive/rachel rotated data/"+x)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
files_list = [] 
for x in os.listdir("/content/drive/My Drive/Research/outerrim/CF3-OuterRim-CF3grouplike-cz-rand"):
  if x[0] == str('C'): 
    files_list.append(x)
  else:
    pass

In [ ]:
#scale of bulkflows used for this set is 150 MPC 
bulkflows = np.load("/content/drive/My Drive/Research/outerrim/CF3-OuterRim-CF3grouplike-cz-rand/true_bulkflows150.npy")
bulkflows[:,0]

array([ -25.39076889,  148.71427005, -144.27017343])

In [ ]:
#using all catalogs and rotating each i amount of times 

galaxy_index = []
path = "/content/drive/My Drive/Research/outerrim/CF3-OuterRim-CF3grouplike-cz-rand"

for x in files_list: 
    data = np.load(path + "/" + x)

    i = 0 
    while i < 1: 
        temp = re.findall(r'\d+', x)
        res = list(map(int, temp))
        bulkflow_ind = res[2]
        galaxy_index.append(bulkflow_ind)

        bulkflow = bulkflows[:,bulkflow_ind] #0 - bulkflow ind

        rshift = data[:,6]
        distmod = 5*np.log(data[:,7]) + 25 
        distmod_err = np.random.normal(0, 0.4, len(distmod))

        glon= data[:,10]*np.pi/180
        glat= data[:,11]*np.pi/180

        #make glon and glat into array of positions on the unit sphere
        pos= np.array([np.cos(glon)*np.cos(glat),np.sin(glon)*np.cos(glat),np.sin(glat)])
        pos=np.transpose(pos)

        alpha= 2*np.pi*np.random.random()  #generate random rotation angles
        gamma= 2*np.pi*np.random.random()
        beta= np.pi*np.random.random()

        r = R.from_euler('xzx', [alpha,beta,gamma]) #create the rotation

        posprime=r.apply(pos)  #apply the rotation to the vectors

        #bulkflows already in cartesian, so no transformation like above? 
        bulkflowprime=r.apply(bulkflow)

        #rotated_bulkflows.append(bulkflowprime)

        glonprime= np.arctan2(posprime[:,1],posprime[:,0])*180/np.pi  #convert back to glon and glat
        glonprime[glonprime<0]= 360+glonprime[glonprime<0]

        glatprime= np.arcsin(posprime[:,2])*180/np.pi

        # add rshift, dist mod, dist mod error, glon, glat, bulk flow to new file 
        with open('/content/drive/My Drive/Rotated new all diff/Rotated Catalog' + str(bulkflow_ind) + '_' + str(i) + '.npy.npz', 'wb') as f:
            #stuff = np.array([rshift,glonprime,glatprime,distmod,distmod_err,bulkflowprime])
            np.savez(f, redshift=rshift, glon=glonprime, glat=glatprime, distm = distmod, distmerr = distmod_err, bulkflow=bulkflowprime)
            f.close()

        i+=1

In [ ]:
#just one catalog, rotate i amount of times
galaxy_index = []
path = "/content/drive/My Drive/Research/outerrim/CF3-OuterRim-CF3grouplike-cz-rand"

x = files_list[0]
data = np.load(path + "/" + x)

i = 0 
while i < 3000: 
    temp = re.findall(r'\d+', x)
    res = list(map(int, temp))
    bulkflow_ind = res[2]
    galaxy_index.append(bulkflow_ind)

    bulkflow = bulkflows[:,bulkflow_ind] #0 - bulkflow ind

    rshift = data[:,6]
    distmod = 5*np.log(data[:,7]) + 25 
    distmod_err = np.random.normal(0, 0.4, len(distmod))

    glon= data[:,10]*np.pi/180
    glat= data[:,11]*np.pi/180

    #make glon and glat into array of positions on the unit sphere
    pos= np.array([np.cos(glon)*np.cos(glat),np.sin(glon)*np.cos(glat),np.sin(glat)])
    pos=np.transpose(pos)

    alpha= 2*np.pi*np.random.random()  #generate random rotation angles
    gamma= 2*np.pi*np.random.random()
    beta= np.pi*np.random.random()

    r = R.from_euler('xzx', [alpha,beta,gamma]) #create the rotation

    posprime=r.apply(pos)  #apply the rotation to the vectors

    #bulkflows already in cartesian, so no transformation like above? 
    bulkflowprime=r.apply(bulkflow)

    #rotated_bulkflows.append(bulkflowprime)

    glonprime= np.arctan2(posprime[:,1],posprime[:,0])*180/np.pi  #convert back to glon and glat
    glonprime[glonprime<0]= 360+glonprime[glonprime<0]

    glatprime= np.arcsin(posprime[:,2])*180/np.pi

    # add rshift, dist mod, dist mod error, glon, glat, bulk flow to new file 
    with open('/content/drive/My Drive/Rotated/Rotated Catalog' + str(bulkflow_ind) + '_' + str(i) + '.npy.npz', 'wb') as f:
        #stuff = np.array([rshift,glonprime,glatprime,distmod,distmod_err,bulkflowprime])
        np.savez(f, redshift=rshift, glon=glonprime, glat=glatprime, distm = distmod, distmerr = distmod_err, bulkflow=bulkflowprime)
        f.close()

    i+=1

In [ ]:
files_list[1] #didn't work???

'CF3-OuterRim-CF3grouplike-cz-rand-box-254.npy'